In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

model_name = "Jean-Baptiste/camembert-ner"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

ner_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)

text = "Paul Martin habite à Lyon et travaille chez ACME."
print(ner_pipeline(text))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Device set to use cpu


[{'entity_group': 'PER', 'score': np.float32(0.9861423), 'word': 'Paul Martin', 'start': 0, 'end': 11}, {'entity_group': 'LOC', 'score': np.float32(0.9981527), 'word': 'Lyon', 'start': 20, 'end': 25}, {'entity_group': 'ORG', 'score': np.float32(0.9924844), 'word': 'ACME', 'start': 43, 'end': 48}]


##Connexion à Snowflake.
##Récupérer les textes depuis RAW_DATA.FISCAL_DOCUMENTS_RAW.
##Charger CamemBERT-NER et préparer le pipeline NER.

In [ ]:
!pip install "snowflake-connector-python[pandas]" --quiet

In [ ]:
import snowflake.connector
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import json
import re
from getpass import getpass

# 1) Connexion Snowflake
ACCOUNT   = "TTPCHZB-OP31538"
USER      = "zineb"
PASSWORD  = getpass("Mot de passe Snowflake :")
WAREHOUSE = "COMPUTE_WH"
DATABASE  = "PII_ANONYMIZATION"

conn = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema="RAW_DATA",
)
cur = conn.cursor()

# 2) Charger les documents
cur.execute("""
    SELECT DOC_ID, DOCUMENT_TEXT
    FROM RAW_DATA.FISCAL_DOCUMENTS_RAW
    ORDER BY DOC_ID
""")
rows = cur.fetchall()
docs = [{"DOC_ID": r[0], "DOCUMENT_TEXT": r[1]} for r in rows]
print("Docs chargés depuis Snowflake:", len(docs))

# 3) Charger CamemBERT-NER
model_name = "Jean-Baptiste/camembert-ner"

print("Chargement CamemBERT-NER (Colab)...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

ner_pipeline = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)

print("CamemBERT-NER prêt.")


Mot de passe Snowflake :··········
Docs chargés depuis Snowflake: 50
Chargement CamemBERT-NER (Colab)...


Device set to use cpu


CamemBERT-NER prêt.


#Fonction de détection PII avec CamemBERT + regex

In [ ]:
def detect_pii_ner(text, max_length=512):
    # Détecte PII avec NER
    entities = ner_pipeline(text[:max_length])
    out = []
    for e in entities:
        out.append({
            "type": e["entity_group"],
            "text": e["word"],
            "score": float(e["score"]),
            "source": "ner"
        })
    return out


def detect_fiscal_pii(text):
    # Détecte PII avec règles (SIRET, montants, etc.)
    pii = []

        # SIRET : 14 chiffres, souvent écrit en blocs "xxx xxx xxx xxxxx"
    siret_pattern = r"\b(?:\d{3}\s\d{3}\s\d{3}\s\d{5}|\d{14})\b"
    for m in re.finditer(siret_pattern, text):
      pii.append({"type": "SIRET", "text": m.group().strip(), "score": 0.99, "source": "regex"})

    # SIREN : 9 chiffres, souvent écrit "xxx xxx xxx"
    siren_pattern = r"\b(?:\d{3}\s\d{3}\s\d{3}|\d{9})\b"
    for m in re.finditer(siren_pattern, text):
      pii.append({"type": "SIREN", "text": m.group().strip(), "score": 0.9, "source": "regex"})

    # Code postal (5 chiffres)
    for m in re.finditer(r"\b\d{5}\b", text):
        pii.append({"type": "CODE_POSTAL", "text": m.group(), "score": 0.95, "source": "regex"})

    # Adresses postales
    for m in re.finditer(r"\b\d+\s+(?:rue|avenue|boulevard|place)\s+[\w\s]+", text, re.I):
        pii.append({"type": "ADRESSE", "text": m.group(), "score": 0.9, "source": "regex"})

    # Dates de naissance (format JJ/MM/AAAA ou J/M/AAAA)
    date_naiss_pattern = r"\b(\d{1,2}/\d{1,2}/\d{4})\b"
    for m in re.finditer(date_naiss_pattern, text):
       pii.append({"type": "DATE_NAISSANCE", "text": m.group().strip(), "score": 0.95,"source": "regex"})

    # Âges
    for m in re.finditer(r"\b(\d+)\s*(?:ans?|age)\b", text, re.I):
        age = int(m.group(1))
        if 18 <= age <= 110:
            pii.append({"type": "AGE", "text": m.group(), "score": 0.9, "source": "regex"})

    # Montants
    for m in re.finditer(r"(\d+(?:[.,]\d{3})*(?:[.,]\d{2})?)\s*(?:€|EUR)", text):
        pii.append({"type": "MONTANT", "text": m.group(), "score": 0.85, "source": "regex"})

    # Emails
    email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
    for m in re.finditer(email_pattern, text):
        pii.append({"type": "EMAIL", "text": m.group(), "score": 0.9, "source": "regex"})

    # Téléphone français
    phone_pattern = r"(?:(?:\+|00)33|0)\s*[1-9](?:[\s.-]?\d{2}){4}"
    for m in re.finditer(phone_pattern, text):
        pii.append({"type": "TEL", "text": m.group(), "score": 0.9, "source": "regex"})

    # IBAN (simplifié, FR ou générique SEPA, avec ou sans espaces)
    iban_pattern = r"\b[A-Z]{2}\d{2}(?:\s?[A-Z0-9]{4}){3,7}\b"
    for m in re.finditer(iban_pattern, text):
      pii.append({ "type": "IBAN", "text": m.group().strip(), "score": 0.98, "source": "regex"})

    # NIR (NIR numéro de sécurité sociale)
    nir_pattern = r"\b[12]\d{2}(0[1-9]|1[0-2])(2[AB]|[0-9]{2})\d{3}\d{3}\d{2}\b"
    for m in re.finditer(nir_pattern, text):
        pii.append({"type": "NIR", "text": m.group(), "score": 0.99, "source": "regex"})

    return pii


print("Fonctions de détection prêtes.")


Fonctions de détection prêtes.


#Appliquer à tous les docs et renvoyer dans Snowflake
Pipeline complet sur tous les docs

In [ ]:
results = []

for i, d in enumerate(docs):
    doc_id = d["DOC_ID"]
    text = d["DOCUMENT_TEXT"]

    ner_pii = detect_pii_ner(text)
    rule_pii = detect_fiscal_pii(text)
    all_pii = ner_pii + rule_pii

    seen = set()
    unique = []
    for p in all_pii:
        key = (p["type"], p["text"])
        if key not in seen:
            seen.add(key)
            unique.append(p)

    results.append({
        "DOC_ID": doc_id,
        "PII_DETECTED": json.dumps(unique),
        "PII_COUNT": len(unique)
    })

    if (i + 1) % 10 == 0:
        print(f"{i+1}/{len(docs)} docs traités")

print("Détection PII terminée.")


10/50 docs traités
20/50 docs traités
30/50 docs traités
40/50 docs traités
50/50 docs traités
Détection PII terminée.


Ecrire dans PROCESSED_DATA.PII_DETECTION_TEMP

In [ ]:
# On bascule sur le schéma PROCESSED_DATA
cur.execute("USE SCHEMA PROCESSED_DATA")

# Créer / écraser la table de résultats
cur.execute("""
    CREATE OR REPLACE TABLE PII_DETECTION_TEMP (
        DOC_ID NUMBER,
        PII_DETECTED VARIANT,
        PII_COUNT NUMBER
    )
""")

# Insérer les résultats calculés par CamemBERT + regex
for r in results:
    cur.execute(
        """
        INSERT INTO PII_DETECTION_TEMP (DOC_ID, PII_DETECTED, PII_COUNT)
        SELECT %s, PARSE_JSON(%s), %s
        """,
        (
            r["DOC_ID"],
            r["PII_DETECTED"],
            r["PII_COUNT"],
        ),
    )

conn.commit()

# Vérification
cur.execute("SELECT COUNT(*) FROM PII_DETECTION_TEMP")
print("Lignes dans PII_DETECTION_TEMP:", cur.fetchone()[0])

Lignes dans PII_DETECTION_TEMP: 50
